In [69]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col, isnull, pow, sqrt,regexp_extract,regexp_replace
from pyspark.ml.feature import Imputer
import os

In [70]:
spark = SparkSession.builder.appName("practica").getOrCreate()
spark.conf.set("spark.hadoop.mapreduce.fileoutputcommitter.marksuccessfuljobs", "false")

In [71]:
df_train = spark.read.csv("../data/raw-kaggle/train.csv", header=True, inferSchema=True)

In [72]:
df_train.show()

+----+---------+-------+--------------------+------------------+------+--------------------+--------------------+----+--------------------+--------------------+----------+------+-----------+
|  id|laptop_ID|Company|             Product|          TypeName|Inches|    ScreenResolution|                 Cpu| Ram|              Memory|                 Gpu|     OpSys|Weight|Price_euros|
+----+---------+-------+--------------------+------------------+------+--------------------+--------------------+----+--------------------+--------------------+----------+------+-----------+
| 268|     1002|   Dell|       Inspiron 5567|          Notebook|  15.6|            1366x768|Intel Core i7 750...| 8GB|             1TB HDD|  AMD Radeon R7 M445|Windows 10|2.36kg|      749.0|
| 347|      867|   Asus|X541NA (N4200/4GB...|          Notebook|  15.6|            1366x768|Intel Pentium Qua...| 4GB|             1TB HDD|Intel HD Graphics...|Windows 10|   2kg|      449.0|
| 353|      966|Toshiba|   Portege Z30-C-1CW|

In [73]:
df_train.select("Company").distinct().show()

+---------+
|  Company|
+---------+
|    Razer|
|  Fujitsu|
|   Huawei|
|   Xiaomi|
|       HP|
|     Dell|
|     Vero|
|     Acer|
|     Asus|
|   Lenovo|
| Mediacom|
|  Samsung|
|   Google|
|       LG|
|    Chuwi|
|Microsoft|
|    Apple|
|      MSI|
|  Toshiba|
+---------+



In [74]:
df_train.select("Product").distinct().show()

+--------------------+
|             Product|
+--------------------+
|              XPS 13|
|   Legion Y720-15IKB|
|V310-15ISK (i3-60...|
|       Aspire E5-575|
|       LifeBook A556|
|FX503VM-E4007T (i...|
|       Inspiron 5370|
|FX753VD-GC086T (i...|
|          GS60 Ghost|
|V310-15IKB (i5-72...|
|         ProBook 470|
|      Surface Laptop|
|     Tecra A50-D-11M|
|      Aspire F5-573G|
|          GL62M 7RDX|
|     Tecra A50-D-11D|
|   TravelMate B117-M|
|       Thinkpad E470|
|     ZenBook UX430UA|
|V110-15IKB (i5-72...|
+--------------------+
only showing top 20 rows



In [75]:
df_train.select("TypeName").distinct().show()

+------------------+
|          TypeName|
+------------------+
|            Gaming|
|         Ultrabook|
|2 in 1 Convertible|
|       Workstation|
|          Notebook|
|           Netbook|
+------------------+



In [76]:
df_train.select("Inches").distinct().show()

+------+
|Inches|
+------+
|  15.4|
|  11.6|
|  17.3|
|  13.3|
|  12.5|
|  12.3|
|  18.4|
|  10.1|
|  13.5|
|  13.9|
|  14.0|
|  13.0|
|  15.0|
|  15.6|
|  12.0|
+------+



In [77]:
df_train.select("ScreenResolution").distinct().show()

+--------------------+
|    ScreenResolution|
+--------------------+
|IPS Panel Quad HD...|
| IPS Panel 2560x1440|
|IPS Panel Touchsc...|
|Touchscreen 2560x...|
|IPS Panel Retina ...|
|Full HD / Touchsc...|
|Touchscreen / Ful...|
|IPS Panel Touchsc...|
|Touchscreen 2256x...|
|IPS Panel Quad HD...|
|            1366x768|
|Touchscreen 2400x...|
|           2560x1440|
|           1920x1080|
|IPS Panel 4K Ultr...|
|4K Ultra HD / Tou...|
|IPS Panel Touchsc...|
|Touchscreen 1366x768|
|IPS Panel Full HD...|
|  IPS Panel 1366x768|
+--------------------+
only showing top 20 rows



In [78]:
df_train.select("Cpu").distinct().show()

+--------------------+
|                 Cpu|
+--------------------+
|Intel Core i7 682...|
|Intel Core i5 730...|
|AMD A6-Series 922...|
|Intel Core i5 730...|
|Intel Core i5 1.8GHz|
|Intel Core i7 865...|
|Intel Core M 6Y30...|
|Intel Core i7 855...|
|AMD A12-Series 97...|
|Intel Pentium Dua...|
|Intel Atom x5-Z85...|
|Intel Celeron Dua...|
|Intel Core M M7-6...|
|Intel Core i5 720...|
|Intel Pentium Qua...|
|Intel Core i5 1.6GHz|
|Intel Core i7 7Y7...|
|Intel Core i5 630...|
|AMD A12-Series 97...|
|Intel Core i5 1.3GHz|
+--------------------+
only showing top 20 rows



In [79]:
df_train.select("Ram").distinct().show()

+----+
| Ram|
+----+
|12GB|
|16GB|
| 8GB|
|24GB|
| 2GB|
|32GB|
| 4GB|
| 6GB|
+----+



In [80]:
df_train.select("Memory").distinct().show()

+--------------------+
|              Memory|
+--------------------+
|512GB SSD +  1.0T...|
|           180GB SSD|
|256GB SSD +  2TB HDD|
|            16GB SSD|
|             2TB HDD|
|128GB SSD +  1TB HDD|
|64GB Flash Storag...|
|            32GB HDD|
|           500GB HDD|
|  1TB HDD +  1TB HDD|
|  16GB Flash Storage|
|             1TB HDD|
|  64GB Flash Storage|
|  32GB Flash Storage|
|256GB SSD +  1.0T...|
|256GB SSD +  1TB HDD|
| 128GB Flash Storage|
|            32GB SSD|
|  1TB SSD +  1TB HDD|
|512GB SSD +  1TB HDD|
+--------------------+
only showing top 20 rows



In [81]:
df_train.select("Gpu").distinct().show()

+--------------------+
|                 Gpu|
+--------------------+
|Nvidia GeForce GT...|
|Intel HD Graphics...|
|Nvidia GeForce GT...|
| AMD Radeon R5 M420X|
|Intel HD Graphics...|
|  AMD Radeon R7 M440|
|   AMD Radeon RX 560|
|   AMD Radeon RX 550|
|Nvidia GeForce MX130|
| Nvidia Quadro M500M|
| Nvidia GeForce 960M|
|Nvidia GeForce GT...|
|Intel HD Graphics...|
| Nvidia Quadro M1200|
|  AMD Radeon R7 M360|
|       AMD Radeon R7|
|Intel Iris Pro Gr...|
|Intel HD Graphics...|
|Nvidia GeForce GT...|
|      AMD R17M-M1-70|
+--------------------+
only showing top 20 rows



In [82]:
df_train.select("OpSys").distinct().show()

+------------+
|       OpSys|
+------------+
|  Windows 10|
|       Linux|
|       macOS|
|   Chrome OS|
|     Android|
|Windows 10 S|
|   Windows 7|
|    Mac OS X|
|       No OS|
+------------+



In [83]:
df_train.select("Weight").distinct().show()

+------+
|Weight|
+------+
|   2kg|
| 3.2kg|
|2.02kg|
|1.49kg|
|0.99kg|
|2.23kg|
|1.05kg|
|1.32kg|
|1.17kg|
| 3.6kg|
|1.28kg|
| 2.9kg|
|1.34kg|
| 3.8kg|
|1.63kg|
| 2.3kg|
|1.35kg|
|2.99kg|
|1.10kg|
|2.67kg|
+------+
only showing top 20 rows



In [84]:
df_train.select("Price_euros").distinct().show()

+-----------+
|Price_euros|
+-----------+
|      299.0|
|      769.0|
|    2296.95|
|     850.66|
|    1426.66|
|     344.99|
|     3949.4|
|      598.9|
|    1962.98|
|     618.99|
|      410.8|
|      443.9|
|    1686.64|
|      599.9|
|     252.36|
|     735.07|
|     469.01|
|     817.95|
|      210.8|
|    2198.19|
+-----------+
only showing top 20 rows



In [85]:
df_train = df_train.withColumn("Intel", when(col("Cpu").like("%Intel%"), True).otherwise(False))
df_train = df_train.withColumn("Intel_baja", when(col("Cpu").like("%Intel Core I3%"), True).otherwise(False))
df_train = df_train.withColumn("Intel_media", when(col("Cpu").like("%Intel Core I5%"), True).otherwise(False))
df_train = df_train.withColumn("Intel_alta", when(col("Cpu").like("%Intel Core I7%"), True).otherwise(False))
df_train = df_train.withColumn("Gaming", when(col("TypeName").like("%Gaming%"), True).otherwise(False))
df_train = df_train.withColumn("Ultrabook", when(col("TypeName").like("%Ultrabook%"), True).otherwise(False))
df_train = df_train.withColumn("Notebook", when(col("TypeName").like("%Notebook%"), True).otherwise(False))
df_train = df_train.withColumn("AMD", when(col("Cpu").like("%AMD%"), True).otherwise(False))
df_train = df_train.withColumn("Nvidia", when(col("Gpu").like("%Nvidia%"), True).otherwise(False))
df_train = df_train.withColumn("Nvidia_Geforce", when(col("Gpu").like("%Geforce%"), True).otherwise(False))
df_train = df_train.withColumn("Nvidia_graphics", when(col("Gpu").like("%HD%"), True).otherwise(False))
df_train = df_train.withColumn("Intel_graphics", when(col("Gpu").like("%Intel%"), True).otherwise(False))
df_train = df_train.withColumn("Screen_HD", when(col("Screenresolution").like("%HD%"), True).otherwise(False))
df_train = df_train.withColumn("SSD", when(col("Memory").like("%SSD%"), True).otherwise(False))
df_train = df_train.withColumn("HDD", when(col("Memory").like("%HDD%"), True).otherwise(False))
df_train = df_train.withColumn("HDD+SSD", when(col("Memory").like("%HDD%") & col("Memory").like("%SSD%"), True).otherwise(False))
df_train = df_train.withColumn("Ram", regexp_replace(col("Ram"), "[^0-9]", "").cast("int"))

In [86]:
df_temporal = df_train
df_temporal = df_temporal.withColumn("X_res", regexp_extract(col("Screenresolution"), r'(\d+)x\d+', 1))
df_temporal = df_temporal.withColumn("Y_res", regexp_extract(col("Screenresolution"), r'\d+x(\d+)', 1))

df_train = df_temporal.withColumn("ppi", sqrt(pow(col("X_res"), 2) + pow(col("Y_res"), 2)) / col("Inches"))

In [87]:
df_train.show()

+----+---------+-------+--------------------+------------------+------+--------------------+--------------------+---+--------------------+--------------------+----------+------+-----------+-----+----------+-----------+----------+------+---------+--------+-----+------+--------------+---------------+--------------+---------+-----+-----+-------+-----+-----+------------------+
|  id|laptop_ID|Company|             Product|          TypeName|Inches|    ScreenResolution|                 Cpu|Ram|              Memory|                 Gpu|     OpSys|Weight|Price_euros|Intel|Intel_baja|Intel_media|Intel_alta|Gaming|Ultrabook|Notebook|  AMD|Nvidia|Nvidia_Geforce|Nvidia_graphics|Intel_graphics|Screen_HD|  SSD|  HDD|HDD+SSD|X_res|Y_res|               ppi|
+----+---------+-------+--------------------+------------------+------+--------------------+--------------------+---+--------------------+--------------------+----------+------+-----------+-----+----------+-----------+----------+------+---------+--

In [88]:
columnas_innecesarias = ["X_res", "Y_res", "TypeName", "Company","id", "laptop_ID", "ScreenResolution","Inches","Product","Memory","Opsys","GPU","weight","CPU"]
df_train = df_train.drop(*columnas_innecesarias)

In [89]:
df_train.show()

+---+-----------+-----+----------+-----------+----------+------+---------+--------+-----+------+--------------+---------------+--------------+---------+-----+-----+-------+------------------+
|Ram|Price_euros|Intel|Intel_baja|Intel_media|Intel_alta|Gaming|Ultrabook|Notebook|  AMD|Nvidia|Nvidia_Geforce|Nvidia_graphics|Intel_graphics|Screen_HD|  SSD|  HDD|HDD+SSD|               ppi|
+---+-----------+-----+----------+-----------+----------+------+---------+--------+-----+------+--------------+---------------+--------------+---------+-----+-----+-------+------------------+
|  8|      749.0| true|     false|      false|     false| false|    false|    true|false| false|         false|          false|         false|    false|false| true|  false|100.45466986113651|
|  4|      449.0| true|     false|      false|     false| false|    false|    true|false| false|         false|           true|          true|    false|false| true|  false|100.45466986113651|
|  8|     1460.0| true|     false|      

In [90]:
df_train.dtypes

[('Ram', 'int'),
 ('Price_euros', 'double'),
 ('Intel', 'boolean'),
 ('Intel_baja', 'boolean'),
 ('Intel_media', 'boolean'),
 ('Intel_alta', 'boolean'),
 ('Gaming', 'boolean'),
 ('Ultrabook', 'boolean'),
 ('Notebook', 'boolean'),
 ('AMD', 'boolean'),
 ('Nvidia', 'boolean'),
 ('Nvidia_Geforce', 'boolean'),
 ('Nvidia_graphics', 'boolean'),
 ('Intel_graphics', 'boolean'),
 ('Screen_HD', 'boolean'),
 ('SSD', 'boolean'),
 ('HDD', 'boolean'),
 ('HDD+SSD', 'boolean'),
 ('ppi', 'double')]

In [91]:
df_train.write.option("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false").csv("../data/processed/train/", header=True, mode="overwrite")

archivos = os.listdir("../data/processed/train/")

# Seleccionar el último archivo en la lista
ultimo_archivo = archivos[-1]

# Construir la ruta completa de los archivos
ruta_antiguo = os.path.join("../data/processed/train/", ultimo_archivo)
ruta_nuevo = os.path.join("../data/processed/train/", "train.csv")

# Cambiar el nombre del último archivo
os.rename(ruta_antiguo, ruta_nuevo)

In [92]:
#ahora para test:

df_test = spark.read.csv("../data/raw-kaggle/test.csv", header=True, inferSchema=True)

df_test = df_test.withColumn("Intel", when(col("Cpu").like("%Intel%"), True).otherwise(False))
df_test = df_test.withColumn("Intel_baja", when(col("Cpu").like("%Intel Core I3%"), True).otherwise(False))
df_test = df_test.withColumn("Intel_media", when(col("Cpu").like("%Intel Core I5%"), True).otherwise(False))
df_test = df_test.withColumn("Intel_alta", when(col("Cpu").like("%Intel Core I7%"), True).otherwise(False))
df_test = df_test.withColumn("Gaming", when(col("TypeName").like("%Gaming%"), True).otherwise(False))
df_test = df_test.withColumn("Ultrabook", when(col("TypeName").like("%Ultrabook%"), True).otherwise(False))
df_test = df_test.withColumn("Notebook", when(col("TypeName").like("%Notebook%"), True).otherwise(False))
df_test = df_test.withColumn("AMD", when(col("Cpu").like("%AMD%"), True).otherwise(False))
df_test = df_test.withColumn("Nvidia", when(col("Gpu").like("%Nvidia%"), True).otherwise(False))
df_test = df_test.withColumn("Nvidia_Geforce", when(col("Gpu").like("%Geforce%"), True).otherwise(False))
df_test = df_test.withColumn("Nvidia_graphics", when(col("Gpu").like("%HD%"), True).otherwise(False))
df_test = df_test.withColumn("Intel_graphics", when(col("Gpu").like("%Intel%"), True).otherwise(False))
df_test = df_test.withColumn("Screen_HD", when(col("Screenresolution").like("%HD%"), True).otherwise(False))
df_test = df_test.withColumn("SSD", when(col("Memory").like("%SSD%"), True).otherwise(False))
df_test = df_test.withColumn("HDD", when(col("Memory").like("%HDD%"), True).otherwise(False))
df_test = df_test.withColumn("HDD+SSD", when(col("Memory").like("%HDD%") & col("Memory").like("%SSD%"), True).otherwise(False))
df_test = df_test.withColumn("Ram", regexp_replace(col("Ram"), "[^0-9]", "").cast("int"))

df_temporal = df_test
df_temporal = df_temporal.withColumn("X_res", regexp_extract(col("Screenresolution"), r'(\d+)x\d+', 1))
df_temporal = df_temporal.withColumn("Y_res", regexp_extract(col("Screenresolution"), r'\d+x(\d+)', 1))

df_test = df_temporal.withColumn("ppi", sqrt(pow(col("X_res"), 2) + pow(col("Y_res"), 2)) / col("Inches"))
columnas_innecesarias = ["X_res", "Y_res", "TypeName", "Company","id", "laptop_ID", "ScreenResolution","Inches","Product","Memory","Opsys","GPU","weight","CPU"]
df_test = df_test.drop(*columnas_innecesarias)

In [93]:
df_test.show()

+---+-----+----------+-----------+----------+------+---------+--------+-----+------+--------------+---------------+--------------+---------+-----+-----+-------+------------------+
|Ram|Intel|Intel_baja|Intel_media|Intel_alta|Gaming|Ultrabook|Notebook|  AMD|Nvidia|Nvidia_Geforce|Nvidia_graphics|Intel_graphics|Screen_HD|  SSD|  HDD|HDD+SSD|               ppi|
+---+-----+----------+-----------+----------+------+---------+--------+-----+------+--------------+---------------+--------------+---------+-----+-----+-------+------------------+
| 16| true|     false|      false|     false| false|     true|   false|false| false|         false|           true|          true|     true| true|false|  false| 331.2642361026012|
|  8|false|     false|      false|     false| false|    false|    true| true| false|         false|          false|         false|    false| true|false|  false|100.45466986113651|
|  4| true|     false|      false|     false| false|    false|    true|false| false|         false| 

In [94]:
df_test.dtypes

[('Ram', 'int'),
 ('Intel', 'boolean'),
 ('Intel_baja', 'boolean'),
 ('Intel_media', 'boolean'),
 ('Intel_alta', 'boolean'),
 ('Gaming', 'boolean'),
 ('Ultrabook', 'boolean'),
 ('Notebook', 'boolean'),
 ('AMD', 'boolean'),
 ('Nvidia', 'boolean'),
 ('Nvidia_Geforce', 'boolean'),
 ('Nvidia_graphics', 'boolean'),
 ('Intel_graphics', 'boolean'),
 ('Screen_HD', 'boolean'),
 ('SSD', 'boolean'),
 ('HDD', 'boolean'),
 ('HDD+SSD', 'boolean'),
 ('ppi', 'double')]

In [95]:
df_test.write.option("mapreduce.fileoutputcommitter.marksuccessfuljobs", "false").csv("../data/processed/test/", header=True, mode="overwrite")

archivos = os.listdir("../data/processed/test")

# Seleccionar el último archivo en la lista
ultimo_archivo = archivos[-1]

# Construir la ruta completa de los archivos
ruta_antiguo = os.path.join("../data/processed/test/", ultimo_archivo)
ruta_nuevo = os.path.join("../data/processed/test/", "test.csv")

# Cambiar el nombre del último archivo
os.rename(ruta_antiguo, ruta_nuevo)